# 각 팀의 선발선수들의 최근 WAR을 반영하여 승패을 예측하는 모델을 피클로

In [2]:
#승무패를 처리하는 result_fnc0, 승1패를 처리하는 result_fnc1
from BaseballGameResult import result_fnc0, result_fnc1
#1차 모델링 WAR정보가 없는 경우 0 
import pandas as pd
game_war = pd.read_csv('game_war.csv', encoding='cp949')

game_score = pd.read_csv('game_score.csv', encoding = 'cp949', header=None)
game_score.columns = ['경기', '결과']

df = pd.merge(game_war, game_score , on='경기' )
df['결과'] = df.결과.apply(lambda x: int(x.split(':')[0])-int(x.split(':')[1]) )
#승1패
#df['결과'] = df.결과.apply(result_fnc1)
#승무패
df['결과'] = df.결과.apply(result_fnc0)

y = df.결과
x = df.drop(['경기','결과', 'Unnamed: 0'], axis=1)


from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y)

from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100, max_iter=1000)
lr.fit(x_train, y_train)
pred = lr.predict(x_test)


from sklearn.metrics import classification_report
report = classification_report(y_test, pred)

#overfit
pred_over_fit = lr.predict(x_train)
report_over_fit = classification_report(y_train, pred_over_fit)

print(report)
print(report_over_fit)

              precision    recall  f1-score   support

        away       0.50      0.40      0.45       173
        home       0.60      0.71      0.65       233
        near       0.00      0.00      0.00         8

    accuracy                           0.57       414
   macro avg       0.37      0.37      0.36       414
weighted avg       0.55      0.57      0.55       414

              precision    recall  f1-score   support

        away       0.57      0.42      0.48       569
        home       0.58      0.73      0.65       660
        near       0.00      0.00      0.00        12

    accuracy                           0.58      1241
   macro avg       0.38      0.38      0.38      1241
weighted avg       0.57      0.58      0.57      1241



C:\Users\103-04\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import pickle
with open('KBOpredictor.pickle','wb') as f:
    pickle.dump(lr,f)